In [1]:
import pandas as pd

In [2]:
import regex as re

In [50]:
from itertools import chain

In [48]:
from collections import Counter

In [3]:
import os

In [4]:
def read_gs_file(g_file):
    with open(g_file, encoding='utf-8') as gs:
        text = gs.read()
        return text

In [12]:
gs_files = os.listdir('GS_FILES/')
gs_files = [f'./GS_FILES/{i}' for i in gs_files]

In [13]:
gs_files[0]

'./GS_FILES/4pFk.gs'

In [9]:
len(gs_files)

1967

In [14]:
gs_text = [read_gs_file(i) for i in gs_files]

In [15]:
len(set(gs_text))

395

In [16]:
set_gs_text = set(gs_text)

In [19]:
df_gs_text = pd.DataFrame(data={'SRC_CODE': list(set_gs_text)})

In [25]:
df_gs_text = df_gs_text[df_gs_text['SRC_CODE'].apply(lambda x:len(x)>1)].reset_index(drop=True)

In [52]:
df_gs_text

394

In [32]:
pat_multi_line_comment = r'/\*.*?\*/'
pat_multi_line_comment = re.compile(pat_multi_line_comment, re.DOTALL)
pat_single_line_comment = r'//.*'
pat_single_line_comment = re.compile(pat_single_line_comment)

In [33]:
df_gs_text['CLEAN_01'] = df_gs_text['SRC_CODE'].apply(lambda x: re.sub(pat_multi_line_comment, '', x))

In [34]:
df_gs_text['CLEAN_02'] = df_gs_text['CLEAN_01'].apply(lambda x: re.sub(pat_single_line_comment, '', x))

In [36]:
df_gs_text['CLEAN_03'] = df_gs_text['CLEAN_02'].apply(lambda x: x.strip())

In [43]:
df_gs_text['CLEAN_04'] = df_gs_text['CLEAN_03'].apply(lambda x: re.sub(r'\(.*?\)', '()', x))

In [45]:
print(df_gs_text['CLEAN_04'].iloc[393])

function sendChatMessageOnEdit() {
  var range = SpreadsheetApp.getActiveRange();
  var value = range.getValue();
  var oldValue = e.oldValue;
  var ss = range.getSheet().getParent();

  
  var changeMessage;
  if () {
    changeMessage = Utilities.formatString('changed from "%s" to "%s"',
      oldValue, value);
  } else if () {
    changeMessage = Utilities.formatString();
  } else {
    changeMessage = 'cleared';
  }
  var message = Utilities.formatString(
      'The range %s was %s. <%s|Open spreadsheet>.',
      range.getA1Notation(), changeMessage, ss.getUrl());

  
  
  var webhookUrl = 'ENTER INCOMMING WEBHOOK URL HERE';

  
  
  var url = webhookUrl + '&threadKey=' + ss.getId();

  
  UrlFetchApp.fetch(url, {
    method: 'post',
    contentType: 'application/json',
    payload: JSON.stringify({
      text: message
    })
  });
}


In [47]:
df_gs_text['TOKENS'] = df_gs_text['CLEAN_04'].apply(lambda x: x.split())

In [58]:
dict_tokens_gs = dict(Counter(list(chain(*list(df_gs_text['TOKENS'])))).most_common(350))

In [60]:
gs_tokens, gs_tokens_freq = dict_tokens_gs.keys(), dict_tokens_gs.values()

In [62]:
df_gs_tokens_fequency = pd.DataFrame(data={'TOKEN': gs_tokens, 'FREQ': gs_tokens_freq})

In [65]:
df_gs_tokens_fequency = df_gs_tokens_fequency[df_gs_tokens_fequency['FREQ']>=23]

In [66]:
df_gs_tokens_fequency = df_gs_tokens_fequency.reset_index(drop=True)

In [68]:
# df_gs_tokens_fequency.to_pickle('Apps_Script_Top_300_TOKENS', protocol=4)

In [69]:
df_gs_tokens_fequency

,TOKEN,FREQ
0,=,11389
1,var,8988
2,{,6299
3,},5859
4,(),2339
...,...,...
306,Browser.msgBox();,23
307,"""%s""",23
308,"shapeId,",23
309,ca.setColumnWidth();,23


In [75]:
df_gs_tokens_fequency['TOKEN'] = df_gs_tokens_fequency['TOKEN'].astype('str')

In [83]:
df_gs_tokens_fequency = df_gs_tokens_fequency.reset_index(drop=True)

In [88]:
with open('TOKENS_APPS_SCRIPT.txt', encoding='utf-8', mode='+a') as file:
    for i in list(df_gs_tokens_fequency['TOKEN']):
        file.write(f'{i}\n')

In [91]:
df_gs_tokens_fequency = df_gs_tokens_fequency[df_gs_tokens_fequency['TOKEN'].apply(lambda x: len(x)>2)]

In [92]:
df_gs_tokens_fequency = df_gs_tokens_fequency.reset_index(drop=True)

In [98]:
df_gs_tokens_fequency['TOKEN'.split()].to_csv('TOP_250_TOKENS_APPS_SCRIPT.txt', encoding='utf-8', index=False, header=False)

In [99]:
df_gs_tokens_fequency

,TOKEN,FREQ
0,var,8988
1,function,2210
2,return,1337
3,for,944
4,else,889
...,...,...
241,Browser.msgBox();,23
242,"""%s""",23
243,"shapeId,",23
244,ca.setColumnWidth();,23
